<a href="https://colab.research.google.com/github/galenzo17/AI-personal-test/blob/main/Newcresitrisk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalación de dependencias
!pip install pandas numpy scikit-learn tensorflow seaborn

# Importar librerías necesarias
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import seaborn as sns
import matplotlib.pyplot as plt

# Descargar datos de Kaggle
!pip install kaggle
from google.colab import files
files.upload()  # Subir archivo kaggle.json
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d uciml/german-credit
!unzip german-credit.zip

# Cargar y preparar datos
df = pd.read_csv('german_credit_data.csv')

# Preprocesamiento de datos
# Eliminar valores nulos
df = df.dropna()

# Codificar variables categóricas
le = LabelEncoder()
categorical_columns = ['Sex', 'Housing', 'Saving accounts', 'Checking account', 'Purpose']
for column in categorical_columns:
    df[column] = le.fit_transform(df[column].astype(str))

# Separar features y target
X = df.drop(['Risk'], axis=1)
y = df['Risk']

# Normalizar datos
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Dividir datos en train y test
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Crear modelo de red neuronal
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compilar modelo
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Configurar early stopping
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

# Entrenar modelo
history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stopping],
    verbose=1
)

# Evaluar modelo
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'\nTest accuracy: {test_accuracy:.3f}')

# Visualizar resultados del entrenamiento
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

# Realizar predicciones
y_pred = model.predict(X_test)
y_pred_classes = (y_pred > 0.5).astype(int)

# Generar matriz de confusión
from sklearn.metrics import confusion_matrix, classification_report
cm = confusion_matrix(y_test, y_pred_classes)
sns.heatmap(cm, annot=True, fmt='d')
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

# Imprimir reporte de clasificación
print('\nClassification Report:')
print(classification_report(y_test, y_pred_classes))

# Guardar el modelo
model.save('modelo_riesgo_crediticio.h5')
print('\nModelo guardado como "modelo_riesgo_crediticio.h5"')

# Ejemplo de cómo usar el modelo para nuevas predicciones
def predecir_riesgo(datos_cliente):
    # Asegurarse que los datos estén en el mismo formato que los datos de entrenamiento
    datos_escalados = scaler.transform([datos_cliente])
    prediccion = model.predict(datos_escalados)
    return "Alto Riesgo" if prediccion[0][0] > 0.5 else "Bajo Riesgo"

# Ejemplo de uso
nuevo_cliente = [1, 2, 1, 2, 1000, 12, 4, 1, 2, 24]  # Valores ejemplo
resultado = predecir_riesgo(nuevo_cliente)
print(f'\nPredicción para nuevo cliente: {resultado}')